# CSC/XMM hostless catalog

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import matplotlib.pyplot as plt
# plt.style.use('seaborn')

import seaborn as sns

from scripts.utils import set_mpl, data_path
from scripts.cross_match_scripts import cross_match_data_frames,  add_separation_columns
set_mpl()

%matplotlib inline
from astropy.table import Table

matplotlib settings set


matplotlib settings set
matplotlib settings set


***

In [3]:
ero_df = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14.pkl') 
#ero_df = ero_df.query('DET_LIKE_0>10')
ero_df.sample(5)

,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH
8121,SRGe J105752.3+574501,164.467884,57.750193,148.155089,53.458353,8.651753,6.374282,3.876688e-15,1.455389e-15,15.367008,5.769093,3621.395752,0.0,0.0,0.0,9094,7939,164.469893,57.748903,3.776219,81.895668,0.000005,6.268413e+08,6.270701e+08,0.0,0.0,0.004243,0.001593,32.471348,164.467884,57.750193,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,0,16.907000,16.358999,12.645,11.4,5.3,0.2,1,0,-1,0,0,NaN,63608140360,3.776219,3.093060,4.669260,7.571029,137.718696,45.980512,2.983263e-15,6.234289e+19
830,SRGe J103913.3+591444,159.805616,59.245433,149.029437,50.651825,4.307306,172.099518,2.971454e-14,2.976036e-15,141.066223,14.128373,4337.120605,0.0,0.0,0.0,868,706,159.807624,59.244144,1.727163,90.070641,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.032525,0.003258,18.332632,159.805616,59.245433,855527970348215936,7.412518,1,0,20.920341,-1.0,0,NVSS J103914+591427,NaN,Radio,3.953525,2,0,1,0,16.414000,15.733000,12.709,16.5,9.8,-0.5,1,1,1,1237653616933994764,9220268790243414016,7.508928,57985750851,1.727163,1.539891,2.324608,3.769264,133.610099,45.908941,2.286650e-14,9.246730e+19
6459,SRGe J104505.5+572004,161.272863,57.334520,150.583749,52.471897,7.262778,16.374775,5.224705e-15,1.413552e-15,24.295437,6.573168,4248.253418,0.0,0.0,0.0,6849,5744,161.274872,57.333231,3.135646,135.924515,0.000008,6.268413e+08,6.270701e+08,0.0,0.0,0.005719,0.001547,17.366802,161.272863,57.334520,-1,-1.000000,0,-1,NaN,-1.0,-1,SDSS J104505.70+572005.0,1.744607,QSO,5.798193,1,0,1,1,16.851999,16.016001,12.104,12.7,7.8,1.8,1,2,0,1237655107835592976,0,3.619223,65213179084,3.135646,2.596493,3.919645,6.355556,135.986600,44.703627,4.020615e-15,6.449714e+19
5664,SRGe J104515.9+543953,161.316347,54.664624,154.046510,54.160826,4.794170,44.341747,6.029909e-15,1.079488e-15,48.963295,8.765523,7418.350098,0.0,0.0,0.0,3102,2471,161.318356,54.663335,1.967395,62.073540,0.000005,6.268413e+08,6.270701e+08,0.0,0.0,0.006600,0.001182,26.124142,161.316347,54.664624,-1,-1.000000,0,-1,NaN,-1.0,-1,FIRST J104516.4+543957,NaN,AGN_Candidate,9.181452,1,0,1,0,16.379999,15.990000,12.805,17.7,7.8,-0.3,2,0,-1,0,0,NaN,75956388343,1.967395,1.713948,2.587363,4.195311,137.932585,42.436013,4.640251e-15,9.477807e+19
1123,SRGe J110402.2+593712,166.009195,59.619970,145.049354,52.649974,17.662754,6.309711,2.388492e-14,9.667100e-15,12.091905,4.894036,462.507141,0.0,0.0,0.0,9447,8290,166.011204,59.618681,7.858395,172.443588,0.000036,6.268413e+08,6.270701e+08,0.0,0.0,0.026144,0.010582,2.617682,166.009195,59.619970,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,56614752853,7.858395,6.314555,9.532402,15.456430,137.142421,47.976192,1.838038e-14,6.249406e+19


In [4]:
# desi_lh.gz_pkl number of sources and area
false_dens_arcmin = 2418574 / (41.729 * 3600)
print(false_dens_arcmin)
false_dens_arcsec = false_dens_arcmin / 3600

print(f'{ false_dens_arcsec:.3f}')

FALSE_PROB = .03



def fsource_r(rho: float, f_thresh: float) -> float:
    """
    The function calculates the radius which corresponds to the probability `f_thresh`
    to FIND one or more false sources. 

    Args:
        rho (float): the density of the false sources (in arcmin^{-2}).
        f_thresh (float): probability to FIND one or more false sources.

    Returns:
        float: radius (in arcsec).
    """

    rho = rho / 3600 # arcmin^{-2} -> arcsec^{-2}
    pf_r = np.sqrt(-1 / (rho * np.pi) * np.log(1 - f_thresh))
    
    return pf_r




r_false = fsource_r(false_dens_arcmin, FALSE_PROB)

print(f'Радиус для {FALSE_PROB:.0%} вероятности найти ложный: {r_false:.2f}"')

desi = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')
desi.sample(5)

16.099741453452303
0.004
Радиус для 3% вероятности найти ложный: 1.47"


,release,objid,brickid,ra,dec,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,type,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,ref_cat,ref_id,mjd_max,mjd_min,iso_max,iso_min,desi_id
724026,9011,2775,614517,162.016687,58.887894,0.485058,0.522380,1.106282,2.232466,2.771855,-23.385773,1309.86790,523.25850,164.013090,55.581760,4.473693,1.141639,0.001127,0.000014,23.255793,23.185013,22.379135,21.626337,21.392029,NaN,14.706847,11.095624,6.689989,8.247686,4.721912,2.961658,-0.784989,4.853742,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.402846,57862.257246,2018-03-17 09:40:05.868,2017-04-19 06:10:26.090,9011_614517_2775
121846,9011,1929,619602,160.937988,60.828038,2.165878,2.492594,3.023215,4.914885,5.567292,-64.161580,-334.07065,322.96878,106.974110,49.643944,4.328418,1.052265,0.001357,0.000016,21.627892,21.486126,21.286385,20.769325,20.634730,NaN,NaN,38.923710,25.780476,21.301107,10.225344,5.710926,-2.363319,-1.346178,EXP,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58245.189944,57414.414167,2018-05-07 04:33:31.130,2016-01-27 09:56:24.000,9011_619602_1929
1881181,9011,723,601906,160.554822,54.995848,0.245732,0.336510,0.579304,2.120133,2.542990,7.705823,88.12324,287.37780,154.750890,107.935074,4.664495,1.099955,0.001280,0.000015,23.999174,23.665888,23.083437,21.682680,21.485771,20.282768,17.637203,4.165707,4.186142,6.018497,4.578945,2.667057,0.275684,0.344066,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57901.177199,57784.421163,2017-05-28 04:15:10.000,2017-01-31 10:06:28.512,9011_601906_723
613416,9011,2142,614523,164.879296,59.090186,0.249961,0.266421,0.692335,4.444615,5.447469,-22.784847,76.25155,572.30524,167.075910,55.225487,4.465284,1.058841,0.001052,0.000013,23.980782,23.919548,22.889963,20.879010,20.658650,NaN,17.794310,5.979781,3.443700,5.145008,9.392014,5.605446,-0.738844,0.270280,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57895.176036,57809.421548,2017-05-22 04:13:29.479,2017-02-25 10:07:01.770,9011_614523_2142
1579469,9011,134,604381,164.094724,55.643931,0.334142,0.521602,1.452535,3.973688,2.089772,15.284609,554.65790,234.89801,78.499146,28.282200,4.584729,1.148064,0.001142,0.000014,23.662096,23.187740,22.084105,21.000408,21.698765,19.539154,15.639857,5.121188,4.621379,7.724728,8.508453,2.239144,0.516430,2.097310,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57894.219319,57528.162072,2017-05-21 05:15:49.139,2016-05-20 03:53:23.000,9011_604381_134


# CSC

In [5]:
csc_df = pd.read_pickle(data_path+'csc_all_lh.pkl')
csc_df.sample(5)

,name,ra,dec,err_ellipse_r0,err_ellipse_r1,err_ellipse_ang,significance,likelihood,likelihood_class,conf_flag,dither_warning_flag,extent_flag,pileup_flag,sat_src_flag,streak_src_flag,var_flag,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,flux_aper_m,flux_aper_lolim_m,flux_aper_hilim_m,radec_err,r_98,flux_05_2,flux_05_2_err,secure
119568,2CXO J104413.8+585537,161.057872,58.926998,1.618505,0.913233,120.840292,3.515152,103.114573,TRUE,False,False,False,False,False,False,True,2.292786e-16,8.597946e-17,3.725777e-16,4.574264e-16,2.287132e-16,6.741021e-16,0.759218,2.123647,6.867050e-16,3.746029e-16,True
124169,2CXO J104734.6+591240,161.894230,59.211193,1.346775,1.276786,16.070880,4.062500,40.195839,TRUE,False,False,False,False,False,False,False,3.327670e-16,1.073442e-16,5.474553e-16,9.334733e-16,6.606119e-16,1.191974e-15,0.758166,2.120703,1.266240e-15,4.880392e-16,True
125434,2CXO J104852.1+584520,162.217458,58.755808,2.603051,1.801656,144.048742,3.294118,32.708128,TRUE,False,False,False,False,False,False,False,2.015942e-15,1.403960e-15,2.591925e-15,2.142556e-16,4.657730e-17,3.819339e-16,1.293324,3.617620,2.230198e-15,8.732190e-16,True
114798,2CXO J103410.7+575919,158.544629,57.988670,1.118938,0.913140,131.268707,3.875000,49.136845,TRUE,False,False,False,False,False,False,False,3.529194e-16,1.095267e-16,5.963120e-16,1.495239e-15,1.110102e-15,1.857721e-15,0.590031,1.650405,1.848158e-15,6.310341e-16,True
123378,2CXO J104640.3+592609,161.668291,59.435841,1.080356,0.930034,46.450882,7.946852,216.562926,TRUE,True,False,False,False,False,False,False,9.842096e-15,7.421908e-15,1.226228e-14,3.161412e-15,2.536482e-15,3.786342e-15,0.582384,1.629015,1.300351e-14,3.534924e-15,False


## CSC x ERO cross-match

In [6]:
ero_csc_orig = cross_match_data_frames(ero_df, csc_df,
 'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = 'csc')
 
ero_csc = ero_csc_orig.query("csc_n_near==1 & csc_n_matches==1")

ero_csc = ero_csc.query("csc_secure==True")
print('SECURITY FILTER APPLIED (CSC)')
print('cross-matches: only one csc within 30 arcsec and it is unique for the eROSITA source', ero_csc.shape[0])
assert len(ero_csc) == ero_csc.csc_name.nunique()
ero_csc.sample(5)

cross-match radius 30 arcsec
total matches: 941 out of 9215 x 2329
	 total unique pairs: 868
	 total non-unique pairs (duplicates in df2): 73
SECURITY FILTER APPLIED (CSC)
cross-matches: only one csc within 30 arcsec and it is unique for the eROSITA source 555


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,csc_sep,csc_n_near,csc_n_matches
785,SRGe J104656.2+592105,161.734018,59.351366,147.870452,51.321376,7.944486,13.333480,4.557315e-15,1.350316e-15,21.729992,6.438518,4356.102051,0.0,0.0,0.0,7190,6068,161.736027,59.350077,3.450904,72.666168,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.004988,0.001478,18.606159,161.734018,59.351366,-1,-1.000000,0,-1,NaN,-1.0,-1,CLANS 518,0.75000,Galaxy,2.688427,1,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,1237655368746140019,0,3.161912,57596824360,3.450904,2.840208,4.287556,6.952109,134.722099,46.532355,3.507032e-15,8.751850e+19,2CXO J104656.5+592102,161.735718,59.350819,0.761630,0.755841,23.915171,5.857143,189.530067,TRUE,False,False,False,False,False,False,False,1.633902e-15,1.153343e-15,2.090434e-15,1.216011e-15,8.656352e-16,1.566387e-15,0.438371,1.226190,2.849914e-15,8.275780e-16,True,3.689066,1,1
388,SRGe J104725.8+591026,161.857680,59.173888,148.011317,51.488023,5.467354,42.357834,1.044027e-14,1.873675e-15,49.409794,8.867383,4323.632812,0.0,0.0,0.0,3741,2990,161.859688,59.172599,2.292222,124.091988,0.000005,6.268413e+08,6.270701e+08,0.0,0.0,0.011428,0.002051,21.062069,161.857680,59.173888,-1,-1.000000,0,-1,NaN,-1.0,-1,[VV2010c] J104725.9+591026,1.14000,AGN,5.382649,1,1,1,1,16.374001,15.110,12.551,16.9,16.2,2.3,1,1,1,1237658304890209036,0,4.934519,58248954160,2.292222,1.954616,2.950674,4.784406,134.940233,46.418659,8.034195e-15,8.372468e+19,2CXO J104725.9+591025,161.858190,59.173751,0.996630,0.913265,53.268779,9.882482,385.990186,TRUE,False,False,False,False,False,False,False,5.877728e-15,4.928304e-15,6.827152e-15,3.414720e-15,2.791558e-15,4.037883e-15,0.552257,1.544745,9.292449e-15,1.606075e-15,True,1.062878,1,1
99,SRGe J103123.8+574309,157.849052,57.719110,151.923439,50.788137,3.342528,241.302124,3.355341e-14,3.061917e-15,156.710831,14.300646,4266.874023,0.0,0.0,0.0,827,671,157.851060,57.717820,1.226710,94.017136,0.000004,6.268413e+08,6.270701e+08,0.0,0.0,0.036727,0.003352,19.589476,157.849052,57.719110,854993543273428608,8.094464,1,0,20.346239,-1.0,0,SDSS J103123.57+574310.2,0.51468,Seyfert_1,8.106778,1,1,0,0,15.608000,15.073,11.957,27.1,15.7,1.9,1,1,1,1237658304352223538,7980555123004035072,8.179701,63727312098,1.226710,1.194977,1.803927,2.925000,133.511247,44.073102,2.582066e-14,6.662110e+19,2CXO J103123.5+574310,157.848084,57.719446,0.982539,0.834665,114.929794,10.027661,357.109000,TRUE,False,False,False,False,False,False,False,1.163892e-14,1.019785e-14,1.307998e-14,5.811768e-15,5.031770e-15,6.545884e-15,0.526690,1.473232,1.745068e-14,2.302311e-15,True,2.218144,1,1
548,SRGe J103438.2+575440,158.659142,57.911237,151.270093,51.019096,7.154722,22.036058,7.222934e-15,1.708406e-15,33.924999,8.024117,4290.952637,0.0,0.0,0.0,5308,4333,158.661151,57.909947,3.085487,67.518906,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.007906,0.001870,18.583740,158.659142,57.911237,-1,-1.000000,0,-1,NaN,-1.0,-1,CXCCLASXS J103437.7+575443,1.19100,StarburstG,10.494067,1,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,62991159718,3.085487,2.557862,3.861328,6.260997,133.888987

## CSC x DESI cross-match

In [7]:
csc_desi_all = cross_match_data_frames(ero_csc, desi, colname_ra1 = 'csc_ra',colname_dec1 = 'csc_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')
csc_desi_all

cross-match radius 15 arcsec
total matches: 2509 out of 555 x 2418574
	 total unique pairs: 2509
	 total non-unique pairs (duplicates in df2): 0


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,csc_sep,csc_n_near,csc_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep,desi_n_near,desi_n_matches
2,SRGe J110242.6+594120,165.677384,59.688978,145.179791,52.486946,7.653812,211.181183,1.846204e-13,1.864554e-14,134.683884,13.602254,666.474426,0.0,0.0,0.0,679,556,165.679392,59.687689,3.316714,224.252457,0.000026,6.268413e+08,6.270701e+08,0.0,0.0,0.202084,0.020409,3.928642,165.677384,59.688978,860692071522063104,4.485168,1,0,17.027691,-1.0,0,6C 105938+595728,1.83,BLLac,4.485268,1,1,1,1,13.617000,12.659000,9.803,43.9,47.2,27.9,1,3,1,1237658304891060264,0,3.087692,56364004534,3.316714,2.736290,4.130682,6.697745,136.878443,47.936061,1.420726e-13,6.398279e+19,2CXO J110242.8+594122,165.678600,59.689447,2.291189,2.008557,31.648612,14.313177,629.754672,TRUE,False,False,False,False,False,False,False,1.247590e-13,1.097446e-13,1.397733e-13,9.484644e-14,8.480775e-14,1.048851e-13,1.244794,3.481875,2.196054e-13,2.554235e-14,True,2.779278,1,1,9011,4152,616737,165.678178,59.688774,123.783880,140.066150,194.811190,287.176500,388.446700,1089.463000,2591.550300,53.469475,22.336110,17.090487,1.092097,0.326052,0.001556,0.000019,17.247953,17.120436,16.768284,16.353460,16.025955,14.906816,13.966043,905.142700,661.967900,805.361940,300.109280,221.806760,42.980423,11.174195,PSF,-0.023313,140.88054,-0.008907,94.32313,0.07499,50.308243,G2,860692071522063104,58194.477145,57813.360879,2018-03-17 11:27:05.369,2017-03-01 08:39:39.978,9011_616737_4152,2.540902,7,1
1,SRGe J110242.6+594120,165.677384,59.688978,145.179791,52.486946,7.653812,211.181183,1.846204e-13,1.864554e-14,134.683884,13.602254,666.474426,0.0,0.0,0.0,679,556,165.679392,59.687689,3.316714,224.252457,0.000026,6.268413e+08,6.270701e+08,0.0,0.0,0.202084,0.020409,3.928642,165.677384,59.688978,860692071522063104,4.485168,1,0,17.027691,-1.0,0,6C 105938+595728,1.83,BLLac,4.485268,1,1,1,1,13.617000,12.659000,9.803,43.9,47.2,27.9,1,3,1,1237658304891060264,0,3.087692,56364004534,3.316714,2.736290,4.130682,6.697745,136.878443,47.936061,1.420726e-13,6.398279e+19,2CXO J110242.8+594122,165.678600,59.689447,2.291189,2.008557,31.648612,14.313177,629.754672,TRUE,False,False,False,False,False,False,False,1.247590e-13,1.097446e-13,1.397733e-13,9.484644e-14,8.480775e-14,1.048851e-13,1.244794,3.481875,2.196054e-13,2.554235e-14,True,2.779278,1,1,9011,4143,616737,165.676791,59.691063,-0.015934,0.072536,0.596225,3.677433,6.544262,-131.121550,-1138.750900,749.9

In [8]:
csc_desi_closest = (csc_desi_all
    .loc[csc_desi_all.groupby('csc_name')['desi_sep'].idxmin()]
    )

csc_hostless = csc_desi_closest.query("desi_sep>2*csc_r_98 & desi_sep>2*1.43")
#csc_hostless = csc_hostless.query('csc_sep<20') #why we have added this in the first place?

csc_hostless['csc_ero_flux_ratio'] = csc_hostless['csc_flux_05_2'] / csc_hostless['flux_05-20']


csc_hostless.rename(columns = {'csc_sep':'ero_csc_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

csc_hostless = add_separation_columns(csc_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert csc_hostless.srcname_fin.nunique()==csc_hostless.csc_name.nunique()
assert csc_hostless.srcname_fin.nunique()==csc_hostless.shape[0]


print('hostless csc near chandra sources', csc_hostless.shape[0])


csc_hostless.sample(7)

hostless csc near chandra sources 28


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_radec_err,csc_r_98,csc_flux_05_2,csc_flux_05_2_err,csc_secure,ero_csc_sep,csc_n_near,csc_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep_minimal,desi_n_near,desi_n_matches,csc_ero_flux_ratio,desi_ero_sep
2493,SRGe J104917.5+585133,162.322873,58.859156,148.121898,51.877514,14.623010,6.066594,3.270358e-15,1.241256e-15,15.272972,5.796816,4266.539062,0.0,0.0,0.0,6255,5187,162.324881,58.857866,6.488227,59.591431,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.003580,0.001359,17.621183,162.322873,58.859156,-1,-1.000000,0,-1,NaN,-1.0,-1,CXOSW J104916.7+585133,NaN,X,10.867680,1,1,0,0,17.187,16.554001,12.494,8.2,1.9,-0.3,1,1,0,1237658304353338080,0,9.580064,59415215908,6.488227,5.227825,7.891884,12.796392,135.480425,46.286794,2.516668e-15,8.306805e+19,2CXO J104916.5+585133,162.319141,58.859426,1.900903,0.949590,124.203079,3.411765,27.565163,TRUE,False,False,False,False,False,False,False,0.000000e+00,0.000000e+00,1.721394e-16,3.517709e-16,1.507590e-16,5.427323e-16,0.868100,2.428205,3.517709e-16,3.263496e-16,True,7.014423,1,1,9011,553,613770,162.318576,58.856306,0.324210,0.273390,0.430090,-0.616135,-0.189344,-25.666616,202.229870,371.87580,106.791480,72.192630,4.737421,1.150697,0.001011,0.000013,23.702274,23.894129,23.408320,NaN,NaN,NaN,16.735329,6.252101,2.825211,3.654311,-1.341056,-0.203110,-0.815952,0.715477,REX,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0,58194.402846,57519.165729,2018-03-17 09:40:05.868,2016-05-11 03:58:39.000,9011_613770_553,11.279802,2,1,0.139776,13.008652
2051,SRGe J103520.2+573355,158.834253,57.565203,151.619002,51.302995,8.727764,10.466516,4.667627e-15,1.417703e-15,21.820000,6.627410,4270.769531,0.0,0.0,0.0,7244,6121,158.836262,57.563913,3.811093,153.681015,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.005109,0.001552,19.799236,158.834253,57.565203,-1,-1.000000,0,-1,NaN,-1.0,-1,CXCCLASXS J103520.9+573349,0.75,X,2.400142,1,1,0,1,17.680,16.650999,12.528,6.3,4.4,0.7,1,0,-1,0,0,NaN,64320607603,3.811093,3.120235,4.710282,7.637545,134.251508,44.212833,3.591921e-15,5.738736e+19,2CXO J103520.9+573349,158.837140,57.563679,1.650398,1.276601,11.458660,3.875000,48.279513,TRUE,False,False,False,False,False,False,False,2.667747e-15,1.734035e-15,3.556996e-15,1.574239e-15,1.132347e-15,2.016130e-15,0.852421,2.384348,4.241985e-15,1.432870e-15,True,7.821427,1,1,9011,914,609941,158.835675,57.562204,0.110228,0.502531,1.554247,6.035648,6.055666,21.223442,513.940250,598.09480,147.184770,92.629630,4.490143,1.158340,0.001397,0.

***

# 4XMM DR10

In [9]:
xmm_df = pd.read_pickle(data_path+'xmm_all_lh.pkl')
xmm_df.sample(5)

,srcid,iauname,sc_ra,sc_dec,sc_poserr,sc_det_ml,sc_ep_1_flux,sc_ep_1_flux_err,sc_ep_2_flux,sc_ep_2_flux_err,sc_ep_3_flux,sc_ep_3_flux_err,sc_ep_4_flux,sc_ep_4_flux_err,sc_ep_5_flux,sc_ep_5_flux_err,sc_ep_8_flux,sc_ep_8_flux_err,sc_ep_9_flux,sc_ep_9_flux_err,sc_ep_8_fmin,sc_ep_8_fmin_err,sc_ep_8_fmax,sc_ep_8_fmax_err,sc_hr1,sc_hr1_err,sc_hr2,sc_hr2_err,sc_hr3,sc_hr3_err,sc_hr4,sc_hr4_err,sc_extent,sc_ext_err,sc_ext_ml,sc_chi2prob,sc_var_flag,sc_sum_flag,mjd_first,mjd_last,sc_fvar,sc_fvarerr,n_detections,confused,webpage_url,flux_05_2,flux_05_2_err,sigma,r_98,secure
255432,201475117010196,4XMM J105302.8+573053,163.261870,57.514975,1.570400,9.40949,3.846260e-16,1.128480e-16,1.741520e-16,8.905760e-17,2.708310e-16,1.148020e-16,8.826760e-17,1.498450e-16,1.354010e-15,1.170190e-15,2.462010e-15,1.201000e-15,7.275110e-16,2.371670e-16,2.462010e-15,1.201000e-15,2.462010e-15,1.201000e-15,-0.346199,0.252089,0.297608,0.281745,-0.452062,0.295964,0.474464,0.448288,0.0000,NaN,-0.562826,NaN,NaN,0,52612.199398,52613.345764,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20147511...,4.449830e-16,1.452954e-16,1.113646,3.115033,False
253860,205562114010035,4XMM J105010.4+590813,162.543460,59.137167,1.467180,32.19930,7.259920e-16,6.625690e-16,5.133230e-15,1.428030e-15,6.401820e-15,1.853890e-15,5.266510e-15,3.729160e-15,4.839860e-14,3.061030e-14,6.816550e-14,3.096410e-14,2.099440e-14,3.914260e-15,6.816550e-14,3.096410e-14,6.816550e-14,3.096410e-14,0.802051,0.162945,-0.005457,0.194228,-0.480532,0.255744,0.408494,0.386147,0.0000,NaN,-0.935986,NaN,NaN,0,54790.964190,54791.066424,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,1.153505e-14,2.340123e-15,1.040447,2.910287,True
246335,203032602010008,4XMM J103343.5+574044,158.431509,57.678934,0.334649,1422.58000,4.566360e-15,3.349130e-16,6.454860e-15,3.884510e-16,6.773220e-15,4.373790e-16,7.415680e-15,8.142310e-16,7.435980e-15,2.346860e-15,3.278330e-14,2.637980e-15,2.296580e-14,9.378480e-16,2.098430e-14,3.910260e-15,4.263890e-14,3.573750e-15,0.148541,0.044713,-0.019682,0.042645,-0.403451,0.049238,-0.555438,0.092826,0.0000,NaN,-0.031311,0.086755,False,0,53467.914178,53510.606285,0.342803,0.160228,2,False,http://xmm-catalog.irap.omp.eu/source/20303260...,1.322808e-14,5.849740e-16,0.237316,0.663807,True
241419,207851312015005,4XMM J102154.8+605433,155.478535,60.909337,6.433450,11.95150,3.495510e-15,8.399580e-15,3.371250e-14,1.523330e-14,9.026940e-14,2.790390e-14,5.914080e-14,4.399840e-14,1.360600e-13,1.004960e-13,3.226790e-13,1.145270e-13,1.510390e-11,1.424920e-13,3.226790e-13,1.145270e-13,3.226790e-13,1.145270e-13,0.838995,0.361979,0.376789,0.234876,-0.612768,0.251563,-0.164816,0.509933,23.8423,4.53558,6.063270,NaN,NaN,3,57721.437847,57721.489896,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20785131...,1.239819e-13,3.179121e-14,4.562266,12.761342,False
253547,205562114010002,4XMM J104934.4+591955,162.393710,59.332222,0.506604,1342.69000,2.026570e-14,1.530600e-15,1.845080e-14,1.537230e-15,1.713600e-14,1.617580e-15,2.162480e-14,3.212480e-15,3.457840e-14,8.991590e-15,1.137900e-13,9.997270e-15,6.333080e-14,3.647820e-15,1.137900e-13,9.997270e-15,1.137900e-13,9.997270e-15,-0.111794,0.054929,-0.100473,0.061155,-0.382691,0.074236,-0.445881,0.117228,0.0000,NaN,-2.562620,0.525351,False,0,54790.964190,54791.066424,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,3.558680e-14,2.231511e-15,0.359257,1.004896,True


## XMM x eROSITA cross-match

In [10]:
ero_xmm_orig = cross_match_data_frames(ero_df, xmm_df,
 'RA_fin', 'DEC_fin', 'sc_ra', 'sc_dec', match_radius = 30, df_prefix = 'xmm')
 
ero_xmm = ero_xmm_orig.query("xmm_n_near==1 & xmm_n_matches==1")
ero_xmm = ero_xmm.query("xmm_secure==True")
print('SECURITY FILTER APPLIED (XMM)')

ero_xmm = ero_xmm.query("xmm_secure==True")
print('cross-matches: only one xmm within 30 arcsec and it is unique for the eROSITA source', ero_xmm.shape[0])
assert len(ero_xmm) == ero_xmm.xmm_srcid.nunique()
ero_xmm.sample(5)

cross-match radius 30 arcsec
total matches: 978 out of 9215 x 1791
	 total unique pairs: 944
	 total non-unique pairs (duplicates in df2): 34
SECURITY FILTER APPLIED (XMM)
cross-matches: only one xmm within 30 arcsec and it is unique for the eROSITA source 742


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,xmm_sep,xmm_n_near,xmm_n_matches
764,SRGe J104725.4+585917,161.855762,58.988157,148.232583,51.612788,7.516050,14.498633,5.477152e-15,1.510350e-15,25.760838,7.103669,4296.874512,0.0,0.0,0.0,7651,6521,161.857771,58.986868,3.252999,67.433205,0.000009,6.268413e+08,6.270701e+08,0.0,0.0,0.005995,0.001653,27.566893,161.855762,58.988157,-1,-1.000000,0,-1,NaN,-1.0,-1,CLANS 597,0.250,Galaxy,5.761970,2,1,1,1,17.429001,16.323999,12.416,7.6,5.3,2.4,1,1,1,1237655109446599124,0,6.282481,58936336530,3.252999,2.687039,4.056333,6.577191,135.086343,46.262703,4.214882e-15,8.450479e+19,205541210010062,4XMM J104725.3+585915,161.855758,58.987580,1.935420,10.2473,2.928530e-15,1.613860e-15,8.979680e-16,1.134400e-15,2.066560e-15,1.478520e-15,7.094440e-16,2.109330e-15,3.636210e-15,1.417860e-14,1.252330e-14,1.472770e-14,8.844550e-15,3.588420e-15,1.252330e-14,1.472770e-14,1.252330e-14,1.472770e-14,-0.197951,0.308824,0.362215,0.343515,-0.392655,0.481500,0.260758,0.631403,0.0,NaN,-0.674422,NaN,NaN,0,54754.834421,54754.973692,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20554121...,2.964528e-15,1.863568e-15,1.372499,3.839084,True,2.078437,1,1
526,SRGe J103306.9+581935,158.278748,58.326426,150.939948,50.607719,5.812846,34.635323,8.646793e-15,1.741355e-15,38.856644,7.825240,4105.421387,0.0,0.0,0.0,4489,3617,158.280757,58.325137,2.456600,169.036453,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.009465,0.001906,18.101824,158.278748,58.326426,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,0,17.537001,17.158001,12.496,7.6,3.0,0.8,1,0,-1,0,0,NaN,61412465356,2.456600,2.078132,3.137132,5.086740,133.345080,44.710686,6.654044e-15,6.419411e+19,205016215010018,4XMM J103306.6+581938,158.277709,58.327264,1.857970,34.0473,5.987690e-16,7.894710e-16,2.173870e-15,1.181460e-15,9.054300e-15,2.243700e-15,4.630930e-15,3.218730e-15,2.529500e-15,1.423890e-14,2.442450e-14,1.528230e-14,2.165640e-14,4.179660e-15,2.442450e-14,1.528230e-14,2.442450e-14,1.528230e-14,0.484420,0.419789,0.669281,0.167344,-0.497979,0.173588,-0.636176,0.456654,0.0,NaN,-0.955509,NaN,NaN,0,54244.090359,54244.241262,NaN,NaN,1,False,http://xmm-catalog.irap.omp.eu/source/20501621...,1.122817e-14,2.535752e-15,1.317575,3.685455,True,3.599207,1,1
529,SRGe J103406.9+583405,158.528680,58.568065,150.512648,50.564250,6.053107,27.113605,8.611135e-15,1.852459e-15,38.686905,8.322471,4104.413086,0.0,0.0,0.0,4846,3935,158.530688,58.566776,2.570198,73.906235,0.000009,6.268413e+08,6.270701e+08,0.0,0.0,0.009426,0.002028,17.990505,158.528680,58.568065,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,60503248305,2.570198,2.164027,3.266798,5.296989,133.324659,44.985066,6.626604e-15,6.475556e+19,20

## XMM x DESI cross-match

In [11]:
xmm_desi_all = cross_match_data_frames(ero_xmm, desi, colname_ra1 = 'xmm_sc_ra',colname_dec1 = 'xmm_sc_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')
xmm_desi_all

cross-match radius 15 arcsec
total matches: 3293 out of 742 x 2418574
	 total unique pairs: 3293
	 total non-unique pairs (duplicates in df2): 0


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,xmm_sep,xmm_n_near,xmm_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep,desi_n_near,desi_n_matches
5,SRGe J104117.7+584929,160.323629,58.824642,149.266632,51.126914,2.166339,3391.888916,2.361506e-13,7.471780e-15,1120.011108,35.437035,4332.920898,0.0,0.0,0.0,38,27,160.325638,58.823353,0.454968,97.643723,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.258489,0.008179,18.102430,160.323629,58.824642,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,1,0,15.243,15.129000,12.821,31.5,15.9,-0.2,1,3,0,1237658304889815429,0,2.181367,59543516327,0.454968,0.774481,1.169150,1.895733,134.256532,45.696772,1.817271e-13,7.704875e+19,205562112010003,4XMM J104117.9+584929,160.324685,58.824762,0.817968,742.5780,2.808530e-14,3.382310e-15,4.008580e-14,4.210150e-15,4.761820e-14,4.978040e-15,5.426170e-14,9.520310e-15,8.843220e-15,2.233210e-14,1.909300e-13,2.733320e-14,1.567980e-13,1.062820e-14,1.909300e-13,2.733320e-14,1.909300e-13,2.733320e-14,0.136026,0.075853,0.023020,0.072849,-0.437121,0.081245,-0.896701,0.157718,0.0,NaN,-2.276850,0.425014,False,0,54595.266366,54595.352928,0.012697,1.426367,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,8.770400e-14,6.519681e-15,0.580060,1.622515,True,2.015272,1,1,9011,43,613766,160.324219,58.824639,-0.052349,0.298325,1.900912,2.352417,2.387243,53.370483,2019.365800,671.00964,212.87097,70.456120,2.689567,0.915560,0.001440,0.000018,NaN,23.799402,21.794834,21.570036,21.554535,18.181593,14.236904,-1.356040,4.352592,15.955904,3.857940,2.284231,2.025409,8.467377,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.398524,57851.315637,2018-03-17 09:33:52.450,2017-04-08 07:34:31,9011_613766_43,0.975167,9,1
6,SRGe J104117.7+584929,160.323629,58.824642,149.266632,51.126914,2.166339,3391.888916,2.361506e-13,7.471780e-15,1120.011108,35.437035,4332.920898,0.0,0.0,0.0,38,27,160.325638,58.823353,0.454968,97.643723,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.258489,0.008179,18.102430,160.323629,58.824642,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,1,0,15.243,15.129000,12.821,31.5,15.9,-0.2,1,3,0,1237658304889815429,0,2.181367,59543516327,0.4549

In [12]:
xmm_desi_closest = (xmm_desi_all
    .loc[xmm_desi_all.groupby('xmm_srcid')['desi_sep'].idxmin()]
    )

xmm_hostless = xmm_desi_closest.query("desi_sep>2*xmm_r_98 & desi_sep>2*1.43")
#xmm_hostless = xmm_hostless.query('xmm_sep<20') #why we have added this in the first place?

xmm_hostless['xmm_ero_flux_ratio'] = xmm_hostless['xmm_flux_05_2'] / xmm_hostless['flux_05-20']


xmm_hostless.rename(columns = {'xmm_sep':'ero_xmm_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

xmm_hostless = add_separation_columns(xmm_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.xmm_srcid.nunique()
assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.shape[0]


print('hostless xmm near chandra sources', xmm_hostless.shape[0])


xmm_hostless.sample(7)

hostless xmm near chandra sources 33


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,xmm_srcid,xmm_iauname,xmm_sc_ra,xmm_sc_dec,xmm_sc_poserr,xmm_sc_det_ml,xmm_sc_ep_1_flux,xmm_sc_ep_1_flux_err,xmm_sc_ep_2_flux,xmm_sc_ep_2_flux_err,xmm_sc_ep_3_flux,xmm_sc_ep_3_flux_err,xmm_sc_ep_4_flux,xmm_sc_ep_4_flux_err,xmm_sc_ep_5_flux,xmm_sc_ep_5_flux_err,xmm_sc_ep_8_flux,xmm_sc_ep_8_flux_err,xmm_sc_ep_9_flux,xmm_sc_ep_9_flux_err,xmm_sc_ep_8_fmin,xmm_sc_ep_8_fmin_err,xmm_sc_ep_8_fmax,xmm_sc_ep_8_fmax_err,xmm_sc_hr1,xmm_sc_hr1_err,xmm_sc_hr2,xmm_sc_hr2_err,xmm_sc_hr3,xmm_sc_hr3_err,xmm_sc_hr4,xmm_sc_hr4_err,xmm_sc_extent,xmm_sc_ext_err,xmm_sc_ext_ml,xmm_sc_chi2prob,xmm_sc_var_flag,xmm_sc_sum_flag,xmm_mjd_first,xmm_mjd_last,xmm_sc_fvar,xmm_sc_fvarerr,xmm_n_detections,xmm_confused,xmm_webpage_url,xmm_flux_05_2,xmm_flux_05_2_err,xmm_sigma,xmm_r_98,xmm_secure,ero_xmm_sep,xmm_n_near,xmm_n_matches,desi_release,desi_objid,desi_brickid,desi_ra,desi_dec,desi_flux_g,desi_flux_r,desi_flux_z,desi_flux_w1,desi_flux_w2,desi_flux_w3,desi_flux_w4,desi_flux_ivar_g,desi_flux_ivar_r,desi_flux_ivar_z,desi_flux_ivar_w1,desi_flux_ivar_w2,desi_flux_ivar_w3,desi_flux_ivar_w4,desi_dered_mag_g,desi_dered_mag_r,desi_dered_mag_z,desi_dered_mag_w1,desi_dered_mag_w2,desi_dered_mag_w3,desi_dered_mag_w4,desi_snr_g,desi_snr_r,desi_snr_z,desi_snr_w1,desi_snr_w2,desi_snr_w3,desi_snr_w4,desi_type,desi_parallax,desi_parallax_ivar,desi_pmra,desi_pmra_ivar,desi_pmdec,desi_pmdec_ivar,desi_ref_cat,desi_ref_id,desi_mjd_max,desi_mjd_min,desi_iso_max,desi_iso_min,desi_desi_id,desi_sep_minimal,desi_n_near,desi_n_matches,xmm_ero_flux_ratio,desi_ero_sep
3141,SRGe J105008.1+572515,162.533608,57.420888,149.741222,52.928770,10.038040,8.178158,3.775047e-15,1.294375e-15,17.752651,6.086968,4296.238281,0.0,0.0,0.0,8971,7816,162.535617,57.419598,4.410150,108.242859,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.004132,0.001417,18.963697,162.533608,57.420888,-1,-1.0,0,-1,NaN,-1.0,-1,SWIRE J105008.35+572513.3,NaN,IR,3.178566,2,1,1,1,16.954000,15.793000,11.915,11.1,9.2,3.8,1,0,-1,0,0,NaN,64878468808,4.410150,3.588667,5.417424,8.784149,136.733038,45.139574,2.905046e-15,6.393658e+19,201475101010062,4XMM J105008.3+572513,162.534868,57.420442,0.759941,76.0643,1.577280e-16,9.843990e-17,1.173680e-15,1.919790e-16,1.548710e-15,2.493420e-16,1.551320e-15,5.395490e-16,2.718130e-15,3.309490e-15,9.268160e-15,3.439170e-15,5.472320e-15,5.573310e-16,7.645360e-15,3.785690e-15,4.146940e-14,2.791830e-14,0.625881,0.118344,0.145637,0.103588,-0.424740,0.112401,0.046102,0.224345,0.0,NaN,-0.640798,NaN,NaN,1,52562.275475,52569.307396,NaN,NaN,3,False,http://xmm-catalog.irap.omp.eu/source/20147510...,2.722390e-15,3.146861e-16,0.538910,1.507413,True,2.922590,1,1,9011,916,609949,162.531100,57.418207,0.051111,0.219824,0.785727,2.621036,4.538667,-29.534185,517.591860,435.09240,158.403520,107.883700,4.659102,1.094819,0.001211,0.000015,25.707022,24.130198,22.753647,21.452576,20.856916,NaN,15.714970,1.066116,2.766672,8.161117,5.657494,4.748970,-1.027812,1.974994,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.405780,57784.444599,2018-03-17 09:44:19.410,2017-01-31 10:40:13.339,9011_609949_916,10.865947,2,1,0.937125,10.806199
1492,SRGe J104831.3+584027,162.130517,58.674177,148.451873,51.929634,5.318124,49.360653,1.121837e-14,1.915296e-15,52.546410,8.971169,4279.181641,0.0,0.0,0.0,3547,2826,162.132525,58.672887,2.220788,90.357338,0.000012,6.268413e+08,6.270701e+08,0.0,0.0,0.012280,0.002096,16.800625,162.130517,58.674177,-1,-1.0,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,1,0,1,17.422001,16.499001,12.454,7.7,5.1,0.9,

# Joining CSC and XMM counterparts

In [13]:
csc_ctps = csc_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'DET_LIKE_0', 'pos_r98', 'pos_sigma_2d', 'flux_05-20', 'ero_csc_sep', 'csc_name',   'csc_ra', 'csc_dec', 'csc_flux_05_2', 'csc_r_98', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'csc_ero_flux_ratio']]

xmm_ctps = xmm_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'DET_LIKE_0', 'pos_r98', 'pos_sigma_2d', 'flux_05-20', 'ero_xmm_sep', 'xmm_srcid',   'xmm_sc_ra', 'xmm_sc_dec', 'xmm_flux_05_2', 'xmm_r_98', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'xmm_ero_flux_ratio']]

csc_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)
xmm_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)

In [14]:
print('CSC companions', csc_ctps.shape[0])
print('XMM companions', xmm_ctps.shape[0])
print('Total companions possible', xmm_ctps.shape[0] + csc_ctps.shape[0])

CSC companions 28
XMM companions 33
Total companions possible 61


In [15]:
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] <  5 ]
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] >  1/5 ]

csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] <  5 ]
csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] >  1/5 ]

In [16]:
xmm_ctps = xmm_ctps[['srcname_fin', 'RA_fin', 'DEC_fin', 'pos_sigma_2d', 'flux_05-20', 'DET_LIKE_0', 'desi_id', 'desi_ra', 'desi_dec']]
csc_ctps = csc_ctps[['srcname_fin', 'RA_fin', 'DEC_fin', 'pos_sigma_2d', 'flux_05-20', 'DET_LIKE_0','desi_id', 'desi_ra', 'desi_dec']]
xmm_ctps['x_ray_det'] = 'xmm'
csc_ctps['x_ray_det'] = 'csc'
print('After Flux filter')
print('CSC companions', csc_ctps.shape[0])
print('XMM companions', xmm_ctps.shape[0])
print('Total companions possible', xmm_ctps.shape[0] + csc_ctps.shape[0])

After Flux filter
CSC companions 20
XMM companions 31
Total companions possible 51


In [17]:
final_ctps = pd.concat([xmm_ctps, csc_ctps])
final_ctps.sort_values(by='srcname_fin', inplace=True)
final_ctps.reset_index(drop=True, inplace=True)


n_ctps = final_ctps.groupby('srcname_fin')['x_ray_det'].transform(lambda x: len(x.unique())) 
final_ctps['n_x_ray_det'] = n_ctps

n_desi_ctps = final_ctps.groupby('srcname_fin')['desi_id'].transform(lambda x: len(x.unique()))
final_ctps['n_desi_ctps'] = n_desi_ctps


final_ctps = final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )')
## final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )') - opposite of this


final_ctps.drop_duplicates(subset=['srcname_fin', 'desi_id'], inplace=True)


final_ctps = add_separation_columns(final_ctps, colname_ra1 = 'RA_fin', colname_dec1 = 'DEC_fin', colname_ra2 = 'desi_ra', colname_dec2 = 'desi_dec', colname = 'sep_ero_desi')



# #save final_ctps_csv to csv
final_ctps_csv = final_ctps[['srcname_fin', 'desi_id']]
final_ctps_csv.rename(columns={'desi_id':'desi_id_true'}, inplace=True)
final_ctps_csv.desi_id_true = 'hostless'
final_ctps_csv.to_csv(data_path+'validation_ctps_ero_desi_lh_hostless.csv', index=False)
print('Final number of hostless', final_ctps.shape[0])

#final_ctps

Final number of hostless 48


In [18]:
#combine with non-hostless list
df_ctps = pd.read_csv(data_path+'validation_ctps_ero_desi_lh_no_hostless.csv')
df_hostless = pd.read_csv(data_path+'validation_ctps_ero_desi_lh_hostless.csv')
df_ctps = pd.concat([df_ctps, df_hostless])

df_ctps.to_csv(data_path+'validation_ctps_ero_desi_lh.csv', index = False)
df_ctps.sample(10)

,srcname_fin,desi_id_true
338,SRGe J104659.3+573056,9011_609947_2315
324,SRGe J104630.3+590830,9011_615260_646
401,SRGe J104900.5+590931,9011_615261_1933
546,SRGe J105436.0+574238,9011_610726_3456
106,SRGe J103442.3+563248,9011_606787_2368
477,SRGe J105053.2+584345,9011_613771_28
388,SRGe J104842.3+585817,9011_614518_211
280,SRGe J104506.8+560034,9011_605185_4073
187,SRGe J104111.8+573849,9011_610719_2427
352,SRGe J104724.3+572435,9011_609947_3301
